In [1]:
%config IPCompleter.greedy=True

# Creación de Catalogo

Se tomara un archivo de excel previamente tratado y se guardara en una base de datos<br>
El Archivo tendra las siguientes columnas:
<ol>
    <li>Nombre.</li>
    <li>Latitud(en formato decimal)</li>
    <li>Lonigtud(en formato decimal)</li>
</ol>

In [2]:
#Bibliotecas que se usaran.
import pandas as pd
import glob
import pyodbc
import sqlalchemy

In [12]:
#Leemos el archivo excel que contiene la tabla catalogo
df = pd.read_excel("Catalogo.xlsx")
#Nos deshacemos la columna index que del dataframe que tambien se guardo en el excel.
df = df.drop(['Unnamed: 0'], axis=1)
#Se crea un nuevo dataframe con los id a usar y se agregan las columnas indicadas con los nuevos nombres.
catalogo = pd.DataFrame({'id': list(range(1,df.size))})
cols1 = ['EstadoUbicacion','NombreEstacion','latitudN','longitudW', 'Altitud', 'FechaInstalacion']
cols2 =['ESTADO', 'NOMBRE','Latitud N', 'Longitud W','Altitud (m)','Instalación en sitio'] 
catalogo[cols1] = df[cols2]

In [13]:
catalogo.head()

,id,nombre,latitudN,longitudW,Altitud,FechaInstalacion
0,1,"CALVILLO, AGUASCALIENTES",21.849444,-102.712222,1618,2004-11-15
1,2,"PRESA 50 ANIV. , AGUASCALIENTES",22.189306,-102.464972,2063.4,2012-12-05
2,3,"SIERRA FRIA, AGUASCALIENTES",22.269778,-102.608778,2976.31,2012-12-08
3,4,"BAHÍA DE LOS ANGELES, BAJA CALIFORNIA",28.896389,-113.560278,10,2000-04-06
4,5,"CATAVIÑA, BAJA CALIFORNIA",29.727222,-114.719167,514,2004-11-10


In [14]:
#Se crea una ruta de conexion a la base de datos llamada clima, la cual se encuentra en el servidor DESKTOP-HQI9VLS
#Utilizamos la autentificacion de windows para conectarnos a este.
server = 'DESKTOP-HQI9VLS'
database = 'clima'
engine = sqlalchemy.create_engine('mssql+pyodbc://'+server+'/'+database+'?trusted_connection=yes&driver=SQL+Server')

In [15]:
#importamos los tipos para no tener para facilitar su uso
from sqlalchemy.types import VARCHAR,Float,Integer

In [17]:
#Utilizando el metodo to_sql guardamos el contenido del DataFrame utilizando la ruta a la base de datos que se creo anteriormente
#Solo especificamos el tipo para nombre, de lo contrario se guardara como varchar(max), los demas los guarda
#de acuerdo con el tipo que tiene en el dataframe
catalogo.to_sql(name='Catalogo', 
                con = engine, 
                index=False, 
                if_exists= 'replace',
                dtype= {'nombre' : VARCHAR(length=255)})
#Se especifica que la columna id sea NOT NULL para despues usarla como llave primaria
with engine.connect() as con:
    con.execute('ALTER TABLE Catalogo alter column id int NOT NULL;')
    con.execute('ALTER TABLE Catalogo ADD PRIMARY KEY (id);')